In [1]:
# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


/home/alexi/Documents/xMD


In [2]:




amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



In [3]:
settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False


config
topology/BPTI


In [4]:
# list config dir 

configs = os.listdir(settings.config)

configs = [config if ("_" in config and ".mdp" in config) else None for config in configs]
configs = [config for config in configs if config is not None]

print(sorted(configs))


mdps = ['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp']

file_names = ["5PTI_APO_amber99_H_solv_ions.gro", "5PTI_APO_amber99_H.top", "5PTI_APO_amber99_H.itp"]

['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp', '7_prod_10ps_out.mdp', 'md_short.mdp']


In [5]:
# specify ARGS: -P, -R, -N

for rep in range(1,5+1):

    md = xMD(settings, 'BPTI_genvel4', "5PTI", rep)

    # md.check_args()
    md.create_directory_structure(overwrite=True)
    md.run_experiment(config_files=mdps,topology_files=file_names)
    save_path = md.save_experiment()

Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel4
Trial directory logs:  logs/MD/5PTI/BPTI_genvel4
Trial directory data:  data/MD/5PTI/BPTI_genvel4
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel4
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel4
Creating directory:  R_1 Creating directory:  R_2 Creating directory:  R_3 

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx grompp -f config/1_minim.mdp -c data/MD/5PTI/BPTI_genvel4/R_1/5PTI_APO_amber99_H_solv_ions.gro -p data/MD/5PTI/BPTI_genvel4/R_1/5PTI_APO_amber99_H.top -o data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_0.tpr -r data/MD/5PTI/BPTI_genvel4/R_1/5PTI_APO_amber99_H_solv_ions.gro -maxwarn 1 -v

Ignoring obsolete mdp entry 'ns-type'

NOTE 1 [file config/1_minim.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using
  verlet-buffer-tolerance.


NOTE 2 [file config/1_minim.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 3 [file config/1_minim.mdp]:
  Setting pcoupl from 'Parrinello-Rahman' to 'no'. Pressure coupling does
  not apply to steep.

Generating 1-4 interactions: f

Trajectory number:  1
checking input for internal consistency...

Setting the LD random seed to -64095637

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Setting gen_seed to -1076766869

Velocities were taken from a Maxwell distribution at 300 K

Reading position restraint coords from data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_0.gro

The center of mass of the position restraint coord's is  2.955  2.975  2.938

The center of mass of the position restraint coo


Back Off! I just backed up data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_1.log to data/MD/5PTI/BPTI_genvel4/R_1/#APO_amber99_5PTI_1.log.1#
Reading file data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_1.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 100, rlist from 0.8 to 0.859

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 20 OpenMP threads 


Back Off! I just backed up data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_1.xtc to data/MD/5PTI/BPTI_genvel4/R_1/#APO_amber99_5PTI_1.xtc.1#

Back Off! I just backed up data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_1.edr to data/MD/5PTI/BPTI_genvel4/R_1/#APO_amber99_5PTI_1.edr.1#
starting mdrun 'TRYPSIN INHIBITOR'
200000 steps,    100.0 ps.
step 16900: timed with pme grid 52 

Trajectory number:  2
checking input for internal consistency...

Setting the LD random seed to -541066537

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Setting gen_seed to -1611686177

Velocities were taken from a Maxwell distribution at 300 K

Reading position restraint coords from data/MD/5PTI/BPTI_genvel4/R_1/APO_amber99_5PTI_1.gro

The center of mass of the position restraint coord's is  2.964  2.982  2.947

The center of mass of the position restraint co

...

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx trjconv -f data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_0.xtc -s data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_0.tpr -pbc mol -center -o data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_0-mol.xtc


-------------------------------------------------------
Program:     gmx trjconv, version 2023.2
Source file: src/gromacs/commandline/cmdlineparser.cpp (line 271)
Function:    void gmx::CommandLineParser::parse(int*, char**)

Error in user input:
Invalid command-line options
  In command-line option -f
    File 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_0.xtc' does not exist
    or is not accessible.
    The file could not be opened.
      Reason: No such file or directory
      (call to fopen() returned error code 2)

For more information and tips for troubleshooting, please check the GROMACS
w

Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 1:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_4.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_4.tpr', '-pbc', 'mol', '-center', '-

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3.tpr, VERSION 2023.2 (single precision)
Reading file data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_3.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 elements
Group     7 (    MainChain+H) has   289 elements
Group     8 (      SideChain) has   603 elements
Group     9 (    SideChain-H) has   221 elements
Group    10 (    Prot-Masses) has   892 elements
Group    11 (    non-Protein) has 19553 elements
Group    12 (          Water) has 19509 elements
Group    13 (            SOL) has 19509 elements
Group    14 (      non-Water) has   936 element

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6.tpr', '-dump', '0', '-o', 'data/MD/5PTI/BPTI_genvel4/R_5/APO_amber99_5PTI_6-nojump.pdb']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjc

Last frame        100 time 1000.000    ->  frame     99 time  990.000      
 ->  frame    100 time 1000.000      
Last written: frame    100 time 1000.000


GROMACS reminds you: "There's no way you can rely on an experiment" (Gerrit Groenhof)



In [6]:
def parse_gromacs_energy_log(filepath):
    """
    Parse a GROMACS log file to extract energy information at each step into a DataFrame.
    """
   # read the file
    with open(filepath, 'r') as f:
        log = f.readlines()
    
    # find the start of the energy table

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            # find:            Step           Time
            if len(split)==2 and split[0] == "Step" and split[1] == "Time":
                start = i
                break
            #find end

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            if len(split)>1 and split[0] == "Statistics":
                end = i
                break

        # extract the table
        table = log[start+1:end]
    # extract the headers
    df = pd.DataFrame()

    return df


log_file_path = "data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log"
# Parse the log file again with the updated function
energy_df_updated = parse_gromacs_energy_log(log_file_path)


FileNotFoundError: [Errno 2] No such file or directory: 'data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log'